In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import random
import os, pickle
from tensorboardX import SummaryWriter
torch.manual_seed(1)

DATA_PATH = os.environ['DATA_PATH']
USE_CUDA = torch.cuda.is_available()

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 텐서보드 포트 설정
port = pickle.load(open("port.info","rb"))[os.getcwd().split("/")[-2]]

# 텐서보드 데이터 파일 초기화
try:
    shutil.rmtree('runs/')
except:
    pass

## TODO 

1. cat_dog data loader를 불러오시오
2. 아래 설계도를 보고 모델을 선언하시오(모든 Convolution 연산 뒤에는 BachNorm과 ReLU를 적용하시오)
3. 모델 선언 후 배치사이즈를 2로 사용하여 Sanity Check를 해보시오
4. 본 모델은 복습을 목적으로 하는 모델일뿐 실제로는 성능이 안좋을 수도 있습니다. 실제 훈련시켜보시고 모델을 수정해보세요.

<img src="./image_samples/cnn_example_2.png"></img>

In [9]:
class Residual(nn.Module):
    def __init__(self,in_channels,out_channels=None,reduce_grid=False):
        super(Residual,self).__init__()
        
        if out_channels:
            self.increase_depth = True
        else:
            self.increase_depth = False
            out_channels = in_channels
            
        if reduce_grid:
            first_stride = 2
        else:
            first_stride = 1
        
        self.residual_layer = nn.Sequential(
                                                    nn.Conv2d(in_channels,out_channels,3,first_stride,1),
                                                    nn.BatchNorm2d(out_channels),
                                                    nn.ReLU(),
                                                    nn.Conv2d(out_channels,out_channels,3,1,1),
                                                    nn.BatchNorm2d(out_channels),
                                                    nn.ReLU(),
                                                )
        if self.increase_depth:
            self.conv1_layer = nn.Sequential(
                                                            nn.Conv2d(in_channels,out_channels,1,first_stride),
                                                            nn.BatchNorm2d(out_channels),
                                                            nn.ReLU(),
                                                            )
            
    def forward(self,inputs):
        residual = self.residual_layer(inputs) # F(x)
        if self.increase_depth:
            inputs = self.conv1_layer(inputs)
        return F.relu(residual+inputs) # F.relu(F(x)+x)
    
class Inception(nn.Module):
    
    def __init__(self,in_ch,mid_ch13,out_ch13,mid_ch15,out_ch15,out_ch_pool_conv,auxiliary=False):
        super(Inception,self).__init__()
        
        # Conv2d(input_channels,output_channels,kernel_size,stride,padding)
        
        self.conv13 = nn.Sequential(nn.Conv2d(in_ch,mid_ch13,kernel_size=1,stride=1),
                                                    nn.BatchNorm2d(mid_ch13),
                                                    nn.ReLU(),
                                                    nn.Conv2d(mid_ch13,out_ch13,kernel_size=3,stride=1,padding=1),
                                                    nn.BatchNorm2d(out_ch13),
                                                    nn.ReLU())
        
        self.conv15 = nn.Sequential(nn.Conv2d(in_ch,mid_ch15,kernel_size=1,stride=1),
                                                    nn.BatchNorm2d(mid_ch15),
                                                    nn.ReLU(),
                                                    nn.Conv2d(mid_ch15,out_ch15,kernel_size=3,stride=1,padding=1),
                                                    nn.BatchNorm2d(out_ch15),
                                                    nn.ReLU(),
                                                    nn.Conv2d(mid_ch15,out_ch15,kernel_size=3,stride=1,padding=1),
                                                    nn.BatchNorm2d(out_ch15),
                                                    nn.ReLU())
        
        self.pool_conv1 = nn.Sequential(nn.MaxPool2d(3,stride=1,padding=1),
                                                        nn.Conv2d(in_ch,out_ch_pool_conv,kernel_size=1,stride=1),
                                                        nn.BatchNorm2d(out_ch_pool_conv),
                                                        nn.ReLU())
    
    def forward(self,inputs,train=False):
        conv13_out = self.conv13(inputs)
        conv15_out = self.conv15(inputs)
        pool_conv_out = self.pool_conv1(inputs)
        outputs = torch.cat([conv13_out,conv15_out,pool_conv_out],1) # depth-wise concat

        return outputs

In [13]:
# TODO

class CNN(nn.Module):
    
    def __init__(self):
        super(CNN,self).__init__()
        
        self.layer1 = nn.Sequential(
                                                # 3,112,112 -> 64,56,56
                                                nn.Conv2d(3,64,7,2,3),
                                                nn.BatchNorm2d(64),
                                                nn.ReLU(),
                                                nn.MaxPool2d(3,2)
                                                )
        self.residual = nn.Sequential(
                                                # 64,28,28 -> 128,14,14
                                                Residual(64,128,reduce_grid=True),
                                                # 128,14,14 -> 256,7,7
                                                Residual(128,256,reduce_grid=True)
                                                )
        # 256,7,7 -> 200,7,7
        self.inception = Inception(256,128,64,128,32,104)
        self.avg_pooling = nn.AvgPool2d(7,7) # 200,1,1
        self.fc = nn.Linear(200,2)
        
    def forward(self,inputs):
        outputs = self.layer1(inputs)
        outputs = self.residual(outputs)
        print(outputs.size())
        outputs = self.inception(outputs)
        outputs = self.avg_pooling(outputs) # B,200,1,1 -> B,200
        outputs = self.fc(outputs.view(ouputs.size(0),-1))
        
        return outputs

In [14]:
BATCH_SIZE = 2
model = CNN()
writer = SummaryWriter(comment="-custom-cnn")

In [15]:
sample_x = Variable(torch.randn(BATCH_SIZE,3,112,112))

In [16]:
preds = model(sample_x)

torch.Size([2, 256, 7, 7])


RuntimeError: Given groups=1, weight[32, 128, 3, 3], so expected input[2, 32, 7, 7] to have 128 channels, but got 32 channels instead

In [20]:
writer.add_graph(model,(sample_x,))
writer.close()

In [21]:
!tensorboard --logdir runs --port 6006

TensorBoard 0.4.0rc3 at http://dsksd-tf:6006 (Press CTRL+C to quit)
^C


In [42]:
STEP = 10
BATCH_SIZE = 32
LR = 0.01
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.1)
model = CNN(2)

In [43]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

# train
train_dataset = vdatasets.ImageFolder(
        DATA_PATH+"cat_dog/train/",
        transforms.Compose([
            transforms.Resize(120),
            transforms.CenterCrop(112),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32, 
                                           shuffle=True,
                                           num_workers=2)

In [ ]:
model.train()
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(train_loader):
        inputs,targets = batch
        preds = model(Variable(inputs))
        loss = loss_function(preds,Variable(targets))
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
        
        if i % 10==0:
            print("mean loss : %.3f" % np.mean(losses))
            losses=[]